<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>
import موارد لازم
</h1>
<p>
یک تعدادی از موارد که ایمپورت شده اند در تمرین دوباره فراخوانی شده اند از طرفی ممکن است برخی استفاده نشده باشند چرا که یک سری از سل های این تمرین پاک شده اند.
</p>
</div>

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import re
import unicodedata
import urllib3
import shutil
import zipfile
import itertools
from google.colab import drive


2.4.1


In [ ]:
import numpy as np
import pickle

## Neural Machine Translation

In this assignment you are going to practice sequence to sequence models.

You will build a Neural Machine Translation (NMT) model to translate human readable dates ("۲۰ دی ۱۳۹۹") into machine readable dates ("1399-10-20").

Let's import needed modules.

The dataset contains 20000 data for training. Run the code below, to load and see first 10 samples.

<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>
آپلود دیتاست ها
</h1>
</div>

In [ ]:
# Path to 'TrainSet' file
trainSetFile = "TrainSet.pickle"

# Openning the file, and load contents
with open(trainSetFile, 'rb') as file:
    trainSet = pickle.load(file)

# Samples
print("Number of samples in train set:", len(trainSet), "\n")
for i in range(10):
    print("Human Input:", trainSet[i][0], "\t\tMachine Readable:", trainSet[i][1])

Number of samples in train set: 20000 

Human Input: شنبه مرداد ۹ ۱۳۷۸ 		Machine Readable: 1378-05-09
Human Input: ۱۰ ۱۱ ۹۹ 		Machine Readable: 1399-11-10
Human Input: ۱۰.۰۹.۵۰ 		Machine Readable: 1350-09-10
Human Input: تیر ماه 28 هزار و سیصد و هفتاد 		Machine Readable: 1370-04-28
Human Input: پنج شنبه فروردین ماه 26 هزار و سیصد و هفتاد و پنج 		Machine Readable: 1375-01-26
Human Input: ۳/۷/۶۳ 		Machine Readable: 1363-03-07
Human Input: مرداد ۲۲ هزار و سیصد و شصت و هشت 		Machine Readable: 1368-05-22
Human Input: هشتم مهر ۱۳۸۸ 		Machine Readable: 1388-07-08
Human Input: هشت آذر هزار و سیصد و هفتاد و نه 		Machine Readable: 1379-09-08
Human Input: فروردین ماه ۱ ۱۳۶۱ 		Machine Readable: 1361-01-01


In [ ]:
# Path to 'ValidationSet' file
validationSetFile = "ValidationSet.pickle"

# Openning the file, and load contents
with open(validationSetFile, 'rb') as file:
    validationSet = pickle.load(file)
print("Number of samples in validation set:", len(validationSet), "\n")


# Path to 'TestSet' file
testSetFile = "TestSet.pickle"

# Openning the file, and load contents
with open(testSetFile, 'rb') as file:
    testSet = pickle.load(file)
print("Number of samples in test set:", len(testSet))


Number of samples in validation set: 5000 

Number of samples in test set: 5000


### Preprocess File

<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>
پیش پردازش
</h1>
<p>
با توجه به این که شبکه را چگونه می خواهیم بسازیم باید ابتدا پیش پردازشی روی داده ها انجام دهیم تا داده ها برای پردازش در شبکه آماده شوند.
<br>
ما می توانیم به دو صورت شبکه را بسازیم:
</p>
<ul>
  <li>با پردازش لغات (حروفی که با space از یک دیگر جدا شده اند)</li>
  <li>با پردازش حروف</li>
</ul>
<p>
ما در این تمرین ابتدا شبکه ای می سازیم مبتنی بر کلمات، سپس شبکه ای مبتنی بر حروف می سازیم.<br> در این قسمت پیش پردازشی برای شبکه مبتنی برکلمات را مشاهده می کنیم:
</p>

</div>

In [ ]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

In [ ]:
# For non-english characterset translations such as Hindi, Russian, etc. we keep unicode. 
def preprocess_sentence(s):
    s = unicode_to_ascii(s.lower().strip())
    s = s.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    s = re.sub(r"([?.!,¿])", r" \1 ", s)
    s = re.sub(r'[" "]+', " ", s)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    #s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
    s = re.sub(r"-", " ", s)
    s = s.rstrip().strip()
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    s = '<start> ' + s + ' <end>'
    return s

In [ ]:
preprocess_sentence("تیر ماه 28 هزار و سیصد و هفتاد")

'<start> تیر ماه 28 هزار و سیصد و هفتاد <end>'

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
چون شبکه مبتنی بر کلمه است و مبنای تشخیص کلمات(space) است بنابراین در خروجی انسانی مجموعه داده ها چون فرم بصورت 1384-01-5 است یک کلمه حساب می شود ما برای رفع این مشکل - را از میان اعداد حذف کردیم و بنابراین خروجی شامل پنج کلمه شد که کلمه اول و آخر start و end است.
</p>
</div>

In [ ]:
preprocess_sentence('1378-05-09')

'<start> 1378 05 09 <end>'

### Create Dataset

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
تابع tokenize را برای تبدیل همه واژگان به یک فهرست واژگان، می سازیم. <br>
 که از توابع قبلا تعریف شده در کتابخانه ها استفاده می کنند.
 <br>
</p>
</div>

In [ ]:
def tokenize(input):
   tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
   tokenizer.fit_on_texts(input)
   sequences = tokenizer.texts_to_sequences(input)
   sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
   return  sequences, tokenizer

In [ ]:
def max_len(tensor):
    #print( np.argmax([len(t) for t in tensor]))
    return max( len(t) for t in tensor)

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>با استفاده از توابع تعریف شده در بالا
مجموعه داده های ترین و ولید و تست را در این قسمت  می سازیم.
</p>
</div>

In [ ]:
h=[]
m=[]
da=[]
for i in trainSet:
   h1,m1= i
   if h is not None:
      h.append(preprocess_sentence(h1))
      m.append(preprocess_sentence(m1))
      da.append((h1, m1))

In [ ]:
#total samples
print("Total Samples : ", len(da))

Total Samples :  20000


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
داده های ترین
</p>
</div>

In [ ]:
# Tokenize each word into index and return the tokenized list and tokenizer
X , X_tokenizer = tokenize(h)
Y, Y_tokenizer = tokenize(m)

Tx = max_len(X)
Ty = max_len(Y)  

In [ ]:
print("Max length human sequence denoted as Tx : ", Tx)
print("Max length machine sequence denoted as Ty: ", Ty)

Max length human sequence denoted as Tx :  14
Max length machine sequence denoted as Ty:  5


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
به هر کلمه با استفاده از تعدا تکرار که در دیتا ست دارد عددی تعلق می گیرد هر چه فراوانی بیشتر عدد کوچکتر است.
</p>
</div>

In [ ]:
X_tokenizer.word_index['<start>'] #'<start>': 2   # tokenize by frequency
input_vocab_size = len(X_tokenizer.word_index)+1  # add 1 for 0 sequence character
output_vocab_size = len(Y_tokenizer.word_index)+ 1
print("input_vocab_size : ", input_vocab_size)
print("output_vocab_size : " ,output_vocab_size)

input_vocab_size :  1227
output_vocab_size :  86


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
داده های ولیدیشن
</p>
</div>

In [ ]:
hv=[]
mv=[]
dav=[]
for i in validationSet:
   h1,m1= i
   if h is not None:
      hv.append(preprocess_sentence(h1))
      mv.append(preprocess_sentence(m1))
      dav.append((h1, m1))

In [ ]:
# Tokenize each word into index and return the tokenized list and tokenizer
Xv , Xv_tokenizer = tokenize(hv)
Yv, Yv_tokenizer = tokenize(mv)
#X_train,  X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

Txv = max_len(Xv)
Tyv = max_len(Yv)  

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
داده های تست
</p>
</div>

In [ ]:
ht=[]
mt=[]
dat=[]
for i in testSet:
   h1,m1= i
   if h is not None:
      ht.append(preprocess_sentence(h1))
      mt.append(preprocess_sentence(m1))
      dat.append((h1, m1))

In [ ]:
# Tokenize each word into index and return the tokenized list and tokenizer
Xt , Xt_tokenizer = tokenize(ht)
Yt, Yt_tokenizer = tokenize(mt)

Txt = max_len(Xt)
Tyt= max_len(Yt)  

### Model Parameters

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
تعریف پارامتر ها :
<ul>
  <li>بچ سایز: در درس متوجه شدیم که خوب است مینی بچ هایی تعریف شود و همه داده ها را هم زمان به مدل نشان ندهیم برای تعریف این مینی بچ ها بهتر است عدد مضربی از 2 باشد بنابر این کاندیدای ما 8 و 16 و 32 و 64 و 128 است چرا که اعداد بزرگتر و کوچکتر از این مقادیر مناسب نیستند. اگر عدد کوچکتر بگیریم شبیه خیلی عدد کم است و مقادیر 2 و 4 تایی برای داده های بسیار حجیم مثل تصاویر سه بعدی مناسب است. مقادیر بزرگتر از 128 هم برای داده های بسیار کم حجم مناسب است البته ممکن است مدل در آن شرایط هم خوب کار کند اما نیاز داریم برای این بررسی چندین بار دیگر مدل را ترین کنیم که با توجه به این که زمان بر است ما فرصت این کار را نداریم.
  عدد 32 را که ما بین این اعداد بود را انتخاب کردیم و یک بار هم مدل را بری عدد 64 ترین کردیم تفاوت چشم گیر نبود اما نتایج برای عدد 32 بهتر بود  و بنا بر این ما عدد 32 را انتخاب کردیم. اما لازم به ذکر است که روش مناسب تر این است که برای اعداد ذکر شده هر کدام چند بار مدل را ترین کنیم و نتیجه را با میانگین گیری مشخص کنیم. البته باید توجه داشته باشیم که تغییر پایپر پارامتر های دیگر نیز این نتایج تاثیر می گذارد.  </li>
  <li>
  امبددینگ سایز : برای این مورد هیچ مقدار مشابه در پروژه های قبلی ندیدم و برای همین مقدار 256 که برای کاربرد ترجمه بود را امتحان کردم و جواب داد ولی آن چه که واضح است اینجا کاملا کاربرد متفاوت است و ابعاد باید متفاوت و کمتر باشند به همین دلیل عدد 10 و 20 را امتحان کردیم ولی نتایج روی سه مجموعه (آموزشی ، ولید و تست) به مقدار قابل ملاحظه ای بدتر بود. بنابر این همان عدد 256 را گرفتیم. البته عدد 100 را قبلا امتحان کردیم و نتایج خوبی داشت ولی چون نتایج اش پاک کردیم، تصمیم گرفتم که مقادیری که نتایج اش را دارم قرار دهم.
  </li>

  <li>rnn_units , dense_units: مقدار 1024 خیلی خیلی بزرگ است برای این سوال ولی ما مدل را اجرا کردیم و خطا خیلی زیاد نبود. ولی با توجه به این که به خروجی مرتبط است rnn_units و اینجا خروجی ثابت است (سه کلمه به همراه نقطه شروع و پایان) می توان انتظار داشت عدد 5 هم جواب خوبی برگرداند. با این حال عدد بسیار بزرگی مانند 1024 به مدل درجه آزادی بیشتر و محاسبات بیشتر تحمیل می کند اما محدودیتی برای مدل ایجاد نمی کند.
  اما وقتی عدد کوچک تر بگیریم در تعداد اپوچ کمتر ولی زمان بیشتر به دقت مناسب می رسد
  
  در ادامه برای 5 با 10 اپوچ نتیجه را می بینیم برای 1024 با 2 اپوچ
    </li>

</ul>

</p>
</div>

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = len(h)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 256
rnn_units = 100
dense_units = 100
Dtype = tf.float32   #used to initialize DecoderCell Zero state

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, Y)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape) 
print(example_Y.shape)

(64, 14)
(64, 5)


In [ ]:
dataset_v = tf.data.Dataset.from_tensor_slices((Xv, Yv)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_Xv, example_Yv = next(iter(dataset_v))
print(example_Xv.shape) 
print(example_Yv.shape)

(64, 14)
(64, 5)


In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((Xt, Yt)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_Xt, example_Yt = next(iter(dataset_test))
print(example_Xt.shape) 
print(example_Yt.shape)

(64, 14)
(64, 5)


In [ ]:
dataset_v = tf.data.Dataset.from_tensor_slices((Xv, Yv)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_Xv, example_Yv = next(iter(dataset_v))
print(example_Xv.shape) 
print(example_Yv.shape)

(64, 14)
(64, 5)


### Creating Encoder-Decoder Model based on tfa.seq2seq module

### Define Model

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
با توجه به امکانات که کتابخانه های موجود به ما می دهند ما می توانیم در سه سطح متفاوت رگیولاریزیشن را برای جلوگیری از بیش برازش استفاده کنیم: 
<ul>
  <li>kernel_regularizer</li>
  <li>bias_regularizer</li>
  <li>activity_regularizer</li>
</ul>
این موارد از اسمشان نیز مشخص است که رگیلاریزشن را در کدام لایه اعمال می کنند برای جلوگیری از بیش برازش می توانیم از این سه در مدل استفاده کنیم به عنوان مثال در lstm می توان هر سه را مقدار دهی کرد و نتیجه را بررسی کرد ما در مدل این موارد را اعمال نکردیم (به علت محدودیت زمان و سیستم)
اما شما می توانید بررسی کنید.

برای جلوگیری از بیش برازش در مدل در مقاله زیر با توجه به کاربرد توضیحاتی آمده است.
https://arxiv.org/abs/1409.2329 <br>
Recurrent Neural Network Regularization
</p>
<p>
<h3>ساخت مدل<h3>
<dl><ol>
<li>
  <dt>
  شبکه encoder از یک لایه encoder embedding و یک لایه LSTM تشکیل شده است.</dt>
  <dd>
در امبددینگ از رگیلاریزیشن استفاده شده است  که جلوی بیش برازش را می گیرد اما مسئله ای که هست این است که باید این پارامتر را تنظیم کرد باز می توان مقادیر مختلف را به روش های گوناگون (انتخاب رندوم و امتحان اعداد به صورت لوگاریتمی 0.0001 و 0.0001 و 0.001 و 0.01 و 0.1 و 1و  10 و...  => هر بار سعی مکنیم بازه را کوچکتر کنیم تا به مقدار مناسب برسیم ) امتحان کرد. باید توجه شود که مقادیر دیگر هایپر پارامتر می توانند روی این مقادیر تاثیر بگذارند و بنا بر این باید با تغیبر یکی دیگر از آن ها اینجا نیز بار دیگر تنظیم شود. اما چون زمان ما محدود است ما به مقدار tf.keras.regularizers.L2(0.001) برای embeddings matrix که نتیجه خوب ارائه داده بسنده می کنیم.
</dd>
</li>
<li>
  <dt>شبکه decoder  هر دو decoder  و attention mechanism را در بر می گیرد.</dt>
  <dd> برای پیاده سازی attention mechanism را می توان از مدل های مختلف استفاده کرد که دو مورد را در این قسمت امتحان کردیم و نهایتا یکی را انتخاب کردیم اما از آنجا که در درس اشاره نشده به این روش ها ما زمان زیادی رو این موارد نمی گذاریم. در این مدل ما  از LuongAtention و AttentionWrapper استفاده می کنم.
  </dd>
</dl>
</ol>
</p>
</div>

In [ ]:
#ENCODER
class EncoderNetwork(tf.keras.Model):
    def __init__(self,input_vocab_size,embedding_dims, rnn_units ):
        super().__init__()
        self.encoder_embedding = tf.keras.layers.Embedding(input_dim=input_vocab_size,
                                                           output_dim=embedding_dims,embeddings_regularizer=tf.keras.regularizers.L2(0.001))
        self.encoder_rnnlayer = tf.keras.layers.LSTM(rnn_units,return_sequences=True, 
                                                     return_state=True )
    
#DECODER
class DecoderNetwork(tf.keras.Model):
    def __init__(self,output_vocab_size, embedding_dims, rnn_units):
        super().__init__()
        self.decoder_embedding = tf.keras.layers.Embedding(input_dim=output_vocab_size,
                                                           output_dim=embedding_dims) 
        self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
        self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell =  self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler= self.sampler,
                                                output_layer=self.dense_layer)

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.LuongAttention(units, memory = memory, 
                                          memory_sequence_length=memory_sequence_length)
        #return tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)

    # wrap decodernn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state,Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 
        return decoder_initial_state



encoderNetwork = EncoderNetwork(input_vocab_size,embedding_dims, rnn_units)
decoderNetwork = DecoderNetwork(output_vocab_size,embedding_dims, rnn_units)


### Optimizer and Custom Loss Function

In [ ]:
optimizer = tf.keras.optimizers.Adam()

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
تابع loss برای اندازه گیری خطا تعریف می کینم .<br>
</p>
</div>

In [ ]:
def loss_function(y_pred, y):
   
    #shape of y [batch_size, ty]
    #shape of y_pred [batch_size, Ty, output_vocab_size] 
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    loss = tf.reduce_mean(loss)
    return loss


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
برای شروع ، attention_mechanism بدون حافظه آغاز 
می شود.
</p>
</div>

In [ ]:
decoderNetwork.attention_mechanism.memory_initialized

False

### One step of training on a batch

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
تعریف تابع برای ترین کردن
</p>
</div>

In [ ]:

def train_step(input_batch, output_batch,encoder_initial_cell_state):
    #initialize loss = 0
    loss = 0
    with tf.GradientTape() as tape:
        encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
        a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                        initial_state =encoder_initial_cell_state)

        #[last step activations,last memory_state] of encoder passed as input to decoder Network
        
         
        # Prepare correct Decoder input & output sequence data
        decoder_input = output_batch[:,:-1] # ignore <end>
        #compare logits with timestepped +1 version of decoder_input
        decoder_output = output_batch[:,1:] #ignore <start>


        # Decoder Embeddings
        decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

        #Setting up decoder memory from encoder output and Zero State for AttentionWrapperState
        decoderNetwork.attention_mechanism.setup_memory(a)
        decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                           encoder_state=[a_tx, c_tx],
                                                                           Dtype=tf.float32)
        
        #BasicDecoderOutput        
        outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                               sequence_length=BATCH_SIZE*[Ty-1])

        logits = outputs.rnn_output
        #Calculate loss

        loss = loss_function(logits, decoder_output)

    #Returns the list of all layer variables / weights.
    variables = encoderNetwork.trainable_variables + decoderNetwork.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss

### Training


In [ ]:
 import tensorflow as tf
#RNN LSTM hidden and memory state initializer
def initialize_initial_state():
        return [tf.zeros((BATCH_SIZE, rnn_units)), tf.zeros((BATCH_SIZE, rnn_units))]

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در پایین نتایج را برای <br>
rnn_units = 5<br>
dense_units = 5<br>
می بینیم با این که 10 epoch بوده است اما نتایج چندان خوب نیست اما با افزایش تعداد اپوچ نتیجه خیلی خوب هم حاصل می شود.
</p>
</div>

In [ ]:
epochs = 10
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0


    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%20 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))

total loss: 2.3450582027435303 epoch 1 batch 20 
total loss: 2.30682110786438 epoch 1 batch 40 
total loss: 2.3233470916748047 epoch 1 batch 60 
total loss: 2.310060501098633 epoch 1 batch 80 
total loss: 2.3372867107391357 epoch 1 batch 100 
total loss: 2.2338645458221436 epoch 1 batch 120 
total loss: 2.2833166122436523 epoch 1 batch 140 
total loss: 2.2206034660339355 epoch 1 batch 160 
total loss: 2.2111458778381348 epoch 1 batch 180 
total loss: 2.2042877674102783 epoch 1 batch 200 
total loss: 2.172658920288086 epoch 1 batch 220 
total loss: 2.240938663482666 epoch 1 batch 240 
total loss: 2.1468193531036377 epoch 1 batch 260 
total loss: 2.165107011795044 epoch 1 batch 280 
total loss: 2.1230263710021973 epoch 1 batch 300 
total loss: 2.1133880615234375 epoch 2 batch 20 
total loss: 2.114091396331787 epoch 2 batch 40 
total loss: 2.1021080017089844 epoch 2 batch 60 
total loss: 2.0851693153381348 epoch 2 batch 80 
total loss: 2.036990165710449 epoch 2 batch 100 
total loss: 2.04

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در پایین نتایج را برای <br>
rnn_units = 100<br>
dense_units = 100<br>
با 10 اپوچ نتیجه خوبی حاصل شده است. بهتر است تعدا اپوچ ها را کم کرد چرا که روی ترین خطا بشدت کاهش پیدا کرده است. و ممکن است نتیجه اور فیت باشد (باید بررسی شود)
</p>
</div>

In [ ]:
epochs = 10
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0


    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%20 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))

total loss: 1.0135730504989624 epoch 1 batch 20 
total loss: 1.0020941495895386 epoch 1 batch 40 
total loss: 0.9875139594078064 epoch 1 batch 60 
total loss: 0.9205904603004456 epoch 1 batch 80 
total loss: 0.9719870090484619 epoch 1 batch 100 
total loss: 0.8657976388931274 epoch 1 batch 120 
total loss: 0.8795223236083984 epoch 1 batch 140 
total loss: 0.8773382902145386 epoch 1 batch 160 
total loss: 0.7717440128326416 epoch 1 batch 180 
total loss: 0.7342590093612671 epoch 1 batch 200 
total loss: 0.720573365688324 epoch 1 batch 220 
total loss: 0.6764557957649231 epoch 1 batch 240 
total loss: 0.5431921482086182 epoch 1 batch 260 
total loss: 0.44333794713020325 epoch 1 batch 280 
total loss: 0.47422754764556885 epoch 1 batch 300 
total loss: 0.347017765045166 epoch 2 batch 20 
total loss: 0.24115490913391113 epoch 2 batch 40 
total loss: 0.2526314854621887 epoch 2 batch 60 
total loss: 0.2959304451942444 epoch 2 batch 80 
total loss: 0.16182588040828705 epoch 2 batch 100 
total 

In [ ]:
epochs = 2
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0


    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%20 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))

total loss: 0.1784008890390396 epoch 1 batch 20 
total loss: 0.09170575439929962 epoch 1 batch 40 
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_addons/seq2seq/decoder.py", line 485, in body
    next_sequence_lengths,  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/nest.py", line 660, in map_structure
    expand_composites=expand_composites)  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/nest.py", line 659, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "/usr/local/lib/python3.6/dist-packages/tensorflow_addons/seq2seq/decoder.py", line 477, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در پایین نتایج را برای <br>
rnn_units = 1024<br>
dense_units = 1024<br>
با 2 اپوچ نتیجه خوبی حاصل شده است.
اما زمان بسیار بیشتری طول کشیده برای تولید جواب.
ممکن است بتوان گفت اورفیت شده است.
</p>
</div>

In [ ]:
epochs = 2
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0


    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%20 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))

total loss: 2.750474452972412 epoch 1 batch 20 
total loss: 2.544995069503784 epoch 1 batch 40 
total loss: 2.4033634662628174 epoch 1 batch 60 
total loss: 1.9432857036590576 epoch 1 batch 80 
total loss: 1.8823696374893188 epoch 1 batch 100 
total loss: 1.8649293184280396 epoch 1 batch 120 
total loss: 1.7328026294708252 epoch 1 batch 140 
total loss: 1.4836058616638184 epoch 1 batch 160 
total loss: 1.334524154663086 epoch 1 batch 180 
total loss: 1.180592656135559 epoch 1 batch 200 
total loss: 1.1910470724105835 epoch 1 batch 220 
total loss: 0.9840564131736755 epoch 1 batch 240 
total loss: 0.8954675793647766 epoch 1 batch 260 
total loss: 0.6123999953269958 epoch 1 batch 280 
total loss: 0.3736543655395508 epoch 1 batch 300 
total loss: 0.3805537223815918 epoch 2 batch 20 
total loss: 0.09562430530786514 epoch 2 batch 40 
total loss: 0.2182844579219818 epoch 2 batch 60 
total loss: 0.1920340210199356 epoch 2 batch 80 
total loss: 0.15121276676654816 epoch 2 batch 100 
total loss

### Inference
<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
توالی ورودی ایجاد را می کنیم تا به encoder منتقل شود.

ورودی decoder در هر مرحله ، پیش بینی های قبلی آن همراه با hidden state و خروجی encoder است.

وقتی مدل \<end\> را پیش بینی کرد ، پیش بینی را متوقف می کنیم.

و وزن های توجه را برای هر time step ذخیره کنید.
</p>
</div>

In [ ]:

#if trained in same session else use checkpoint variable
#decoder_embedding_matrix = tf.train.load_variable(checkpointdir, 'decoderNetwork/decoder_embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE')
decoder_embedding_matrix = decoderNetwork.decoder_embedding.variables[0] 
print(decoderNetwork.decoder_embedding.variables[0].shape)

(86, 256)


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در این قسمت از نمونهگیر گریدی برای پیش بینی جواب استفاده کردیم.
</p>
</div>

In [ ]:

input_raw="دوازده بهمن هزار و سیصد و هشتاد و دو"
#def inference(input_raw):
input_lines = input_raw.split("\n")
# We have a transcript file containing English-Hindi pairs
# Preprocess X
input_lines = [preprocess_sentence(line) for line in input_lines]
input_sequences = [[X_tokenizer.word_index[w] for w in line.split(' ')] for line in input_lines]
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                maxlen=Tx, padding='post')
inp = tf.convert_to_tensor(input_sequences)
#print(inp.shape)
inference_batch_size = input_sequences.shape[0]
encoder_initial_cell_state = [tf.zeros((inference_batch_size, rnn_units)),
                              tf.zeros((inference_batch_size, rnn_units))]
encoder_emb_inp = encoderNetwork.encoder_embedding(inp)
a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp,
                                                initial_state =encoder_initial_cell_state)


#output_sequences = []
print('a_tx :',a_tx.shape)
print('c_tx :', c_tx.shape)



start_tokens = tf.fill([inference_batch_size],Y_tokenizer.word_index['<start>'])
#print(start_tokens)
end_token = Y_tokenizer.word_index['<end>']

greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
#finished,start_inputs = greedy_sampler.initialize(decoder_embedding_matrix,start_tokens,end_token)
#print(finished.shape, start_inputs.shape)

decoder_input = tf.expand_dims([Y_tokenizer.word_index['<start>']]* inference_batch_size,1)
decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

decoder_instance = tfa.seq2seq.BasicDecoder(cell = decoderNetwork.rnn_cell, sampler = greedy_sampler,
                                            output_layer=decoderNetwork.dense_layer)
decoderNetwork.attention_mechanism.setup_memory(a)
#pass [ last step activations , encoder memory_state ] as input to decoder for LSTM
print("decoder_initial_state = [a_tx, c_tx] :",np.array([a_tx, c_tx]).shape)
decoder_initial_state = decoderNetwork.build_decoder_initial_state(inference_batch_size,
                                                                   encoder_state=[a_tx, c_tx],
                                                                   Dtype=tf.float32)
print("\nCompared to simple encoder-decoder without attention, the decoder_initial_state \
 is an AttentionWrapperState object containing s_prev tensors and context and alignment vector \n ")
print("decoder initial state shape :",np.array(decoder_initial_state).shape)
print("decoder_initial_state tensor \n", decoder_initial_state)

# Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
# One heuristic is to decode up to two times the source sentence lengths.
maximum_iterations = tf.round(tf.reduce_max(Tx) * 2)

#initialize inference decoder

(first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_token,
                             initial_state = decoder_initial_state)
#print( first_finished.shape)
print("\nfirst_inputs returns the same decoder_input i.e. embedding of  <start> :",first_inputs.shape)
print("start_index_emb_avg ", tf.reduce_sum(tf.reduce_mean(first_inputs, axis=0))) # mean along the batch

inputs = first_inputs
state = first_state  
predictions = np.empty((inference_batch_size,0), dtype = np.int32)                                                                             
for j in range(maximum_iterations):
    outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
    inputs = next_inputs
    state = next_state
    outputs = np.expand_dims(outputs.sample_id,axis = -1)
    predictions = np.append(predictions, outputs, axis = -1)
                                                                               

a_tx : (1, 1024)
c_tx : (1, 1024)
decoder_initial_state = [a_tx, c_tx] : (2, 1, 1024)

Compared to simple encoder-decoder without attention, the decoder_initial_state  is an AttentionWrapperState object containing s_prev tensors and context and alignment vector 
 
decoder initial state shape : (6,)
decoder_initial_state tensor 
 AttentionWrapperState(cell_state=[<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[-0.03809679,  0.02568001,  0.01960622, ..., -0.07071487,
        -0.17816448,  0.04117737]], dtype=float32)>, <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[-0.14047505,  0.08054069,  0.09260534, ..., -0.16887328,
        -0.46367982,  0.20955126]], dtype=float32)>], attention=<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, time=<tf.Tensor: shape=(), dtype=int32, numpy=0>, alignments=<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:

print("ورودی:")
print(input_raw)
print("\nخروجی:")
for i in range(len(predictions)):
    line = predictions[i,:]
    seq = list(itertools.takewhile( lambda index: index !=2, line))
    print("-".join( [Y_tokenizer.index_word[w] for w in seq]))

ورودی:
دوازده بهمن هزار و سیصد و هشتاد و دو

خروجی:
1382-11-12


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در این قسمت از نمونه با استفاده از گریدی سرچ تولید شده است.

برای ورودی "دوازده بهمن هزار و سیصد و هشتاد و دو" خروجی 
1382-11-12
را داده است که درست است.
</p>
</div>

### Inference using Beam Search with beam_width = 3

In [ ]:
beam_width = 3
#use with scope /cpu:0 for inferencing
#restore from latest checkpoint for inferencing
input_raw="دوازده بهمن هزار و سیصد و هشتاد و دو"
#input_raw="Wow!"  #checking translation on training set record
#def inference(input_raw):
input_lines = input_raw.split("\n")
# We have a transcript file containing English-Hindi pairs
# Preprocess X
input_lines = [preprocess_sentence(line) for line in input_lines]
input_sequences = [[X_tokenizer.word_index[w] for w in line.split(' ')] for line in input_lines]
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                maxlen=Tx, padding='post')
inp = tf.convert_to_tensor(input_sequences)
#print(inp.shape)
inference_batch_size = input_sequences.shape[0]
encoder_initial_cell_state = [tf.zeros((inference_batch_size, rnn_units)),
                              tf.zeros((inference_batch_size, rnn_units))]
encoder_emb_inp = encoderNetwork.encoder_embedding(inp)
a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp,
                                                initial_state =encoder_initial_cell_state)

start_tokens = tf.fill([inference_batch_size],Y_tokenizer.word_index['<start>'])
#print(start_tokens)
end_token = Y_tokenizer.word_index['<end>']



decoder_input = tf.expand_dims([Y_tokenizer.word_index['<start>']]* inference_batch_size,1)
decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)


#From official documentation
#NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:

#The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
#The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
#The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.
encoder_memory = tfa.seq2seq.tile_batch(a, beam_width)
decoderNetwork.attention_mechanism.setup_memory(encoder_memory)
print("beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] :", encoder_memory.shape)
#set decoder_inital_state which is an AttentionWrapperState considering beam_width
decoder_initial_state = decoderNetwork.rnn_cell.get_initial_state(batch_size = inference_batch_size* beam_width,dtype = Dtype)
encoder_state = tfa.seq2seq.tile_batch([a_tx, c_tx], multiplier=beam_width)
decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 

decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoderNetwork.rnn_cell,beam_width=beam_width,
                                                 output_layer=decoderNetwork.dense_layer)


# Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
# One heuristic is to decode up to two times the source sentence lengths.
maximum_iterations = 5
#tf.round(tf.reduce_max(Tx) * 2)

#initialize inference decoder

(first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_token,
                             initial_state = decoder_initial_state)
#print( first_finished.shape)
print("\nfirst_inputs returns the same decoder_input i.e. embedding of  <start> :",first_inputs.shape)

inputs = first_inputs
state = first_state  
predictions = np.empty((inference_batch_size, beam_width,0), dtype = np.int32)
beam_scores =  np.empty((inference_batch_size, beam_width,0), dtype = np.float32)                                                                            
for j in range(maximum_iterations):
    beam_search_outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
    inputs = next_inputs
    state = next_state
    outputs = np.expand_dims(beam_search_outputs.predicted_ids,axis = -1)
    scores = np.expand_dims(beam_search_outputs.scores,axis = -1)
    predictions = np.append(predictions, outputs, axis = -1)
    beam_scores = np.append(beam_scores, scores, axis = -1)
print(predictions.shape) 
print(beam_scores.shape)                                                                             

beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (3, 14, 100)

first_inputs returns the same decoder_input i.e. embedding of  <start> : (1, 3, 256)
(1, 3, 5)
(1, 3, 5)


In [ ]:
beam_width = 3
#use with scope /cpu:0 for inferencing
#restore from latest checkpoint for inferencing
input_raw="دوازده بهمن هزار و سیصد و هشتاد و دو"
#input_raw="Wow!"  #checking translation on training set record
#def inference(input_raw):
input_lines = input_raw.split("\n")
# We have a transcript file containing English-Hindi pairs
# Preprocess X
input_lines = [preprocess_sentence(line) for line in input_lines]
input_sequences = [[X_tokenizer.word_index[w] for w in line.split(' ')] for line in input_lines]
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                maxlen=Tx, padding='post')
inp = tf.convert_to_tensor(input_sequences)
#print(inp.shape)
inference_batch_size = input_sequences.shape[0]
encoder_initial_cell_state = [tf.zeros((inference_batch_size, rnn_units)),
                              tf.zeros((inference_batch_size, rnn_units))]
encoder_emb_inp = encoderNetwork.encoder_embedding(inp)
a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp,
                                                initial_state =encoder_initial_cell_state)

start_tokens = tf.fill([inference_batch_size],Y_tokenizer.word_index['<start>'])
#print(start_tokens)
end_token = Y_tokenizer.word_index['<end>']



decoder_input = tf.expand_dims([Y_tokenizer.word_index['<start>']]* inference_batch_size,1)
decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)


#From official documentation
#NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:

#The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
#The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
#The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.
encoder_memory = tfa.seq2seq.tile_batch(a, beam_width)
decoderNetwork.attention_mechanism.setup_memory(encoder_memory)
print("beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] :", encoder_memory.shape)
#set decoder_inital_state which is an AttentionWrapperState considering beam_width
decoder_initial_state = decoderNetwork.rnn_cell.get_initial_state(batch_size = inference_batch_size* beam_width,dtype = Dtype)
encoder_state = tfa.seq2seq.tile_batch([a_tx, c_tx], multiplier=beam_width)
decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 

decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoderNetwork.rnn_cell,beam_width=beam_width,
                                                 output_layer=decoderNetwork.dense_layer)


# Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
# One heuristic is to decode up to two times the source sentence lengths.
maximum_iterations = 5
#tf.round(tf.reduce_max(Tx) * 2)

#initialize inference decoder

(first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_token,
                             initial_state = decoder_initial_state)
#print( first_finished.shape)
print("\nfirst_inputs returns the same decoder_input i.e. embedding of  <start> :",first_inputs.shape)

inputs = first_inputs
state = first_state  
predictions = np.empty((inference_batch_size, beam_width,0), dtype = np.int32)
beam_scores =  np.empty((inference_batch_size, beam_width,0), dtype = np.float32)                                                                            
for j in range(maximum_iterations):
    beam_search_outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
    inputs = next_inputs
    state = next_state
    outputs = np.expand_dims(beam_search_outputs.predicted_ids,axis = -1)
    scores = np.expand_dims(beam_search_outputs.scores,axis = -1)
    predictions = np.append(predictions, outputs, axis = -1)
    beam_scores = np.append(beam_scores, scores, axis = -1)
print(predictions.shape) 
print(beam_scores.shape)                                                                             

beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (3, 14, 100)

first_inputs returns the same decoder_input i.e. embedding of  <start> : (1, 3, 256)
(1, 3, 5)
(1, 3, 5)


In [ ]:
print("-----------------")
print("ورودی:")
print(input_raw)
print("-----------------")
print("\nخروجی:")
for i in range(len(predictions)):
    print("---------------------------------------------")
    output_beams_per_sample = predictions[i,:,:]
    score_beams_per_sample = beam_scores[i,:,:]
    for beam, score in zip(output_beams_per_sample,score_beams_per_sample) :
        seq = list(itertools.takewhile( lambda index: index !=2, beam))
        score_indexes = np.arange(len(seq))
        beam_score = score[score_indexes].sum()
        print(" ".join( [Y_tokenizer.index_word[w] for w in seq]), " beam score: ", beam_score)



-----------------
ورودی:
دوازده بهمن هزار و سیصد و هشتاد و دو
-----------------

خروجی:
---------------------------------------------
1382 11 12  beam score:  -0.00053067727
1383 11 12  beam score:  -27.773975
1362 11 12  beam score:  -29.092134


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در این قسمت از نمونه با استفاده از بیم سرچ با اندازه beam 3 تولد شده است.

برای ورودی "دوازده بهمن هزار و سیصد و هشتاد و دو" خروجی 
1382-11-12
را داده است که درست است.

اختلاف امتیاز دو خروجی اشتباه با خروجی درست را مشاهده می کنیم که بسیار زیاد است.

</p>
</div>

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در مقایسه عملکرد دو روش بیم سرچ و گریدی باید گفت:
یا باید هر دو دقت یکسانی داشته باشند یا دقت بیم سرچ باید بهتر باشد.<br>



*  گریدی : در هر گام تنها خروجی که ماکسیمم احتمال را دارد به خانه بعدی می دهیم و ممکن است جواب بهینه را از دست بدهیم 
*  بیم : در هر کام به اندازه پهنای بیم خروجی با احتمال بالا تر را انتخاب می کنیم و به مرحله بعدی می دهیم برای مثال اگر بیم 3 باشد در خروجی اول آن 3 مورد که احتمال بالاتری دارند را به خانه بعدی می دهیم و سپس در مرحله بعد پس از بررسی هر کدام از میان تمام مقدیر تولید شده که 3 برابر قبل است 3 تا باحتمال بیشتر را انتخاب می کنیم و به بعدی می دهیم از اینجا به بعد تمام خروجی ها به اندازه 3 برابر اولین خروجی است که هر بار 3 محتملترین را انتخاب می کنیم.

برای بررسی دقت یک مدل شبکه انکودر و دکودر را در آن مدل باید بررسی کرد از آنجا که ما در این جا شبکه انکودر را یکسان گرفتیم اگر تفاوت وجود داشته باشد باد در دکودر مدل ها بررسی شود.<br>

اگر بطور خاص به دکودر در بیم سرچ نگاه کنیم و  به دنبال بررسی این باشیم که آیا دکودر در این مدل خوب عمل کرده است یا خیر و اگر قرار است برای بهبود مدل کاری انجام دهیم خوب است در دکودر (همان شبکه بیم) تغییر ایجاد کنیم یا باید در شبکه انکودر (RNN) تغییر ایجاد کنیم.

با بررسی p(y|x) و p(y^|x) می توان یافت باید در کدام تغییر ایجاد کرد که در اینجا p(y|x) مقدار مطلوب ما است و p(y^|x) خروجی مدل است 

اگر p(y|x)>p(y^|x) ینی اگر y را به مدل RNN بدهیم احتمال بیشتری می دهد بنابراین در نتیجه شبکه RNN درست بوده و بیم ما اشتباه عمل کرده است اگر برعکس بود و p(y^|x) بیشتر بود نساد می دهد RNN خوب عمل نکرده است و بیم مطابق خروجی RNNعمل کرده است .
</p>
</div>

### Evaluate Loss

In [ ]:
def eval_greedy(input_batch, output_batch,encoder_initial_cell_state, BATCH_SIZE):
    #initialize loss = 0
    loss = 0

    # we can do initialization in outer block
    #encoder_initial_cell_state = encoder.initialize_initial_state()
    encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
    a, h_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                    initial_state =encoder_initial_cell_state)



    decoder_input = output_batch[:,:-1] # ignore <end>
    #compare logits with timestepped +1 version of decoder_input
    decoder_output = output_batch[:,1:] #ignore <start>
    decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)
    decoder_instance = tfa.seq2seq.BasicDecoder(decoderNetwork.rnn_cell, 
                                                greedy_sampler,
                                                decoderNetwork.dense_layer)
    #BasicDecoderOutput

    decoderNetwork.attention_mechanism.setup_memory(a)
    #pass [ last step activations , encoder memory_state ] as input to decoder for LSTM
    
    decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                       encoder_state=[h_tx, c_tx],
                                                                       Dtype=tf.float32)
    outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                           sequence_length=BATCH_SIZE*[Ty-1])
    logits = outputs.rnn_output
    sample_id = outputs.sample_id
    #Calculate loss
    loss = loss_function(logits, decoder_output)
    return loss, sample_id, logits

In [ ]:
def eval_beam(input_batch, output_batch,encoder_initial_cell_state, BATCH_SIZE):
    #initialize loss = 0
    loss = 0

    # we can do initialization in outer block
    #encoder_initial_cell_state = encoder.initialize_initial_state()
    encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
    a, h_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                    initial_state =encoder_initial_cell_state)



    decoder_input = output_batch[:,:-1] # ignore <end>
    #compare logits with timestepped +1 version of decoder_input
    decoder_output = output_batch[:,1:] #ignore <start>
    decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)
    decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoderNetwork.rnn_cell,beam_width=beam_width,
                                                 output_layer=decoderNetwork.dense_layer)
    #BasicDecoderOutput

    decoderNetwork.attention_mechanism.setup_memory(a)
    #pass [ last step activations , encoder memory_state ] as input to decoder for LSTM
    
    decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                       encoder_state=[h_tx, c_tx],
                                                                       Dtype=tf.float32)
    outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                           sequence_length=BATCH_SIZE*[Ty-1])
    
    
    logits = outputs.rnn_output
    sample_id = outputs.sample_id
    #Calculate loss
    loss = loss_function(logits, decoder_output)
    return loss, sample_id

### Evaluation Loss on Entire Test Set

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
از میزان خطایی که داده های ترین در آخرین اپوچ (10) دیدیم مشخص بود که چون خطا خیلی خیلی کم شده اور فیت شده است .

در این قسمت هم با مقایسه خطای روی ولید و تست با ترین کاملا مشخص است که خطا تست با این که حدود 15 درصد است اما نسبت به ترین که حدودا صفر است خیلی خیلی زیاد است.
می توانیم تعدا اپوچ ها را کم کنیم یا این که تعداد یونیت های rnn را مثلا به 50 کاهش دهیم و باز مدل را ترین کنیم و نتایج را ببینیم.
</p>
<p>
ما به علت کمبود وقت دوباره ترین را انجام ندادیم.
</p>
</div>

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((X, Y)).batch(len(X))
a=[]
for (input_batch, output_batch) in dataset_train.take(-1):
    batch_size = len(input_batch)
    print(input_batch.shape)
    encoder_initial_cell_state = [tf.zeros((batch_size, rnn_units)),
                                  tf.zeros((batch_size, rnn_units))]
    loss,_ = eval_greedy(input_batch, output_batch, encoder_initial_cell_state, batch_size)
    loss = tf.reduce_mean(loss)
    print("Training loss {}".format(loss) )

(20000, 14)
Training loss 0.0016933688893914223


In [ ]:
dataset_v = tf.data.Dataset.from_tensor_slices((Xv, Yv)).batch(len(Xv))
a=[]
for (input_batch, output_batch) in dataset_v.take(-1):
    batch_size = len(input_batch)
    print(input_batch.shape)
    encoder_initial_cell_state = [tf.zeros((batch_size, rnn_units)),
                                  tf.zeros((batch_size, rnn_units))]
    loss,_ = eval_greedy(input_batch, output_batch, encoder_initial_cell_state, batch_size)
    a.append((_.numpy()))
    loss = tf.reduce_mean(loss)
    print("Training loss {}".format(loss) )

(5000, 14)
Training loss 15.325642585754395


In [ ]:
dataset_v = tf.data.Dataset.from_tensor_slices((Xv, Yv)).batch(len(Xv))
a=[]
for (input_batch, output_batch) in dataset_v.take(-1):
    batch_size = len(input_batch)
    print(input_batch.shape)
    encoder_initial_cell_state = [tf.zeros((batch_size, rnn_units)),
                                  tf.zeros((batch_size, rnn_units))]
    loss,_ = eval_beam(input_batch, output_batch, encoder_initial_cell_state, batch_size)
    a.append((_.numpy()))
    loss = tf.reduce_mean(loss)
    print("Training loss {}".format(loss) )

(5000, 14)
Training loss 15.325642585754395


In [ ]:
b=[]
for i in a:
  for j in range(5000):
    line = i[j]
    seq = list(itertools.takewhile( lambda index: index !=2, line))
    b.append("-".join( [Y_tokenizer.index_word[w] for w in seq]))
   

In [ ]:
hv1=[]
mv1=[]
dav1=[]
for i in validationSet:
   h1,m1= i
   if h1 is not None:
      hv1.append(h1)
      mv1.append(m1)
      dav1.append((h1, m1))

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((Xt, Yt)).batch(len(Xt))
a=[]
for (input_batch, output_batch) in dataset_test.take(-1):
    batch_size = len(input_batch)
    print(input_batch.shape)
    encoder_initial_cell_state = [tf.zeros((batch_size, rnn_units)),
                                  tf.zeros((batch_size, rnn_units))]
    loss,_ = eval_greedy(input_batch, output_batch, encoder_initial_cell_state, batch_size)
    a.append((_.numpy()))
    loss = tf.reduce_mean(loss)
    print("Training loss {}".format(loss) )

(5000, 14)
Training loss 14.608162879943848


In [ ]:
b=[]
for i in a:
  for j in range(5000):
    line = i[j]
    seq = list(itertools.takewhile( lambda index: index !=2, line))
    b.append("-".join( [Y_tokenizer.index_word[w] for w in seq]))


In [ ]:
ht1=[]
mt1=[]
dat1=[]
for i in testSet:
   h1,m1= i
   if h is not None:
      ht1.append(h1)
      mt1.append(m1)
      dat1.append((h1, m1))

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((Xt, Yt)).batch(len(Xt))
a=[]
for (input_batch, output_batch) in dataset_test.take(-1):
    batch_size = len(input_batch)
    print(input_batch.shape)
    encoder_initial_cell_state = [tf.zeros((batch_size, rnn_units)),
                                  tf.zeros((batch_size, rnn_units))]
    loss,_ = eval_greedy(input_batch, output_batch, encoder_initial_cell_state, batch_size)
    a.append((_.numpy()))
    loss = tf.reduce_mean(loss)
    print("Training loss {}".format(loss) )

(5000, 14)
Training loss 14.608162879943848


<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
بررسی نشان می دهد برای ورودی هایی که عدد بیشتر داشته است خوب عمل نکرده است با این که ما سعی کردیم با حذف . و / تاریخ هایی که اسپیس نداشتند را به سه کلمه تبدیل کنیم باز هم مواردی داشتیم که خوب عمل نکرده بودند.
</p>

</div>

<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>مدل مبتنی بر حروف</h1>
<p>
در قسمت قبل ما مدلی مبتی بر کلمه ساختیم و نتایج را بررسی کردم مدل ما اورفیت شده بود اما با این حال برای داده هایی که خودمان وارد کردیم نتیجه خوبی داشت (البته برای داده های تست تا حدود 15 درصد خطا داشت)
<br>
در این قسمت مدلی مبتنی بر حروف می سازیم و میزان دقت و belu ان را بررسی می کنیم.

</p>

</div>

In [ ]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

#from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
#from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

<div dir='rtl'style="text-align: right;font-family:verdana;">

<h2>پیش پردازش</h2>
</div>

In [ ]:
human_vocab=set()
machine_vocab=set()
dataset=[]
for i in trainSet:
   h,m= i
   if h is not None:
      dataset.append((h, m))
      human_vocab.update(tuple(h))
      machine_vocab.update(tuple(m))
human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], list(range(len(human_vocab) + 2))))
inv_machine = dict(enumerate(sorted(machine_vocab)))
machine = {v: k for k, v in inv_machine.items()}


In [ ]:
machine

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [ ]:
inv_machine

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

In [ ]:
len(dataset)

20000

In [ ]:
vhuman_vocab=set()
vmachine_vocab=set()
vdataset=[]
for i in validationSet:
   h,m= i
   if h is not None:
      vdataset.append((h, m))
      vhuman_vocab.update(tuple(h))
      vmachine_vocab.update(tuple(m))
vhuman = dict(zip(sorted(vhuman_vocab) + ['<unk>', '<pad>'], list(range(len(vhuman_vocab) + 2))))
vinv_machine = dict(enumerate(sorted(vmachine_vocab)))
vmachine = {v: k for k, v in vinv_machine.items()}


In [ ]:
thuman_vocab=set()
tmachine_vocab=set()
tdataset=[]
for i in testSet:
   h,m= i
   if h is not None:
      tdataset.append((h, m))
      thuman_vocab.update(tuple(h))
      tmachine_vocab.update(tuple(m))
thuman = dict(zip(sorted(thuman_vocab) + ['<unk>', '<pad>'], list(range(len(thuman_vocab) + 2))))
tinv_machine = dict(enumerate(sorted(tmachine_vocab)))
tmachine = {v: k for k, v in tinv_machine.items()}

In [ ]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

In [ ]:
def string_to_int(string, length, vocab):
    #string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
بزرگترین ورودی 52 کاراکتر بود اما با در نظر گرفتن ورودی 52 برای مدل دقت مدل به شدت افت کرد بنا بر این ما 50 گرفتیم.

برای 50 نتایج خیلی بهتر بود.

خروجی هم ثابت است (10)
</p>

</div>

In [ ]:
string_to_int('شنبه مرداد ۹ ۱۳۷۸', 52, human)

[24,
 30,
 15,
 31,
 0,
 29,
 21,
 19,
 14,
 19,
 0,
 46,
 0,
 38,
 40,
 44,
 45,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49]

In [ ]:
Tx = 50
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human, machine, Tx, Ty)
vX, vY, vXoh, vYoh = preprocess_data(vdataset, vhuman, vmachine, Tx, Ty)
tX, tY, tXoh, tYoh = preprocess_data(tdataset, thuman, tmachine, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)
print()
print("vX.shape:", vX.shape)
print("vY.shape:", vY.shape)
print("vXoh.shape:", vXoh.shape)
print("vYoh.shape:", vYoh.shape)
print()
print("tX.shape:", tX.shape)
print("tY.shape:", tY.shape)
print("tXoh.shape:", tXoh.shape)
print("tYoh.shape:", tYoh.shape)

X.shape: (20000, 50)
Y.shape: (20000, 10)
Xoh.shape: (20000, 50, 50)
Yoh.shape: (20000, 10, 11)

vX.shape: (5000, 50)
vY.shape: (5000, 10)
vXoh.shape: (5000, 50, 50)
vYoh.shape: (5000, 10, 11)

tX.shape: (5000, 50)
tY.shape: (5000, 10)
tXoh.shape: (5000, 50, 50)
tYoh.shape: (5000, 10, 11)


In [ ]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: شنبه مرداد ۹ ۱۳۷۸
Target date: 1378-05-09

Source after preprocessing (indices): [24 30 15 31  0 29 21 19 14 19  0 46  0 38 40 44 45 49 49 49 49 49 49 49
 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49 49
 49 49]
Target after preprocessing (indices): [ 2  4  8  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>ساخت مدل</h1>
<p>
</p>
</div>

In [ ]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

In [ ]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [ ]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')

In [ ]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model

In [ ]:
mod = model(Tx, Ty, n_a, n_s, len(human), len(machine))

In [ ]:
mod.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 50, 50)]     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 64)]         0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 50, 64)       21248       input_5[0][0]                    
__________________________________________________________________________________________________
repeat_vector_3 (RepeatVector)  (None, 50, 64)       0           s0[0][0]                         
                                                                 lstm_13[0][0]              

In [ ]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>آموزش مدل</h1>

</div>

In [ ]:
s0 = np.zeros((len(trainSet), n_s))
c0 = np.zeros((len(trainSet), n_s))
outputs = list(Yoh.swapaxes(0,1))
s0v = np.zeros((5000, n_s))
c0v = np.zeros((5000, n_s))
outputsv = list(vYoh.swapaxes(0,1))
mod.fit([Xoh, s0, c0], outputs, epochs=40, batch_size=100 ,validation_data=([vXoh, s0v, c0v], outputsv))


Epoch 1/40
200/200 [==============================] - 42s 129ms/step - loss: 17.8905 - dense_17_loss: 0.6439 - dense_17_1_loss: 0.9966 - dense_17_2_loss: 2.4340 - dense_17_3_loss: 2.5127 - dense_17_4_loss: 1.2248 - dense_17_5_loss: 1.6683 - dense_17_6_loss: 2.5530 - dense_17_7_loss: 1.1866 - dense_17_8_loss: 1.9605 - dense_17_9_loss: 2.7100 - dense_17_accuracy: 0.9421 - dense_17_1_accuracy: 0.5774 - dense_17_2_accuracy: 0.1679 - dense_17_3_accuracy: 0.0798 - dense_17_4_accuracy: 0.9021 - dense_17_5_accuracy: 0.1608 - dense_17_6_accuracy: 0.0502 - dense_17_7_accuracy: 0.9288 - dense_17_8_accuracy: 0.1074 - dense_17_9_accuracy: 0.0401 - val_loss: 8.0610 - val_dense_17_loss: 0.0252 - val_dense_17_1_loss: 0.0972 - val_dense_17_2_loss: 0.9519 - val_dense_17_3_loss: 1.6598 - val_dense_17_4_loss: 0.0344 - val_dense_17_5_loss: 0.4146 - val_dense_17_6_loss: 1.7597 - val_dense_17_7_loss: 0.0580 - val_dense_17_8_loss: 1.1333 - val_dense_17_9_loss: 1.9269 - val_dense_17_accuracy: 1.0000 - val_dens

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
در سلول زیر ما خروجی تعدادی از ورودی که خودمان دادیم به مدل را مشاهده می کنیم.
</p>
</div>

In [ ]:
Example_dates = ['۳/۱/۶۱', '۱۳ دی ۸۴', 'چهارشنبه دی ماه 14 هزار و چهارصد', 'مهر ماه ۱ ۱۳۶۱', 
                 "بهمن هزار و سیصد و هشتاد و دو ۱۲", 'دوازده آذر هزار و سیصد و هفتاد', 
                 " سه شهریور هزار و سیصد و هفتاد و پنج", 'هجده آذر 99']
s0 = np.zeros((1, n_s))
c0 = np.zeros((1, n_s))
for example in Example_dates:
    
    source = string_to_int(example, Tx, human)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output)) 

source: ۳/۱/۶۱
output: 1361-03-33
source: ۱۳ دی ۸۴
output: 1384-10-03
source: چهارشنبه دی ماه 14 هزار و چهارصد
output: 1400-10-14
source: مهر ماه ۱ ۱۳۶۱
output: 1361-07-01
source: بهمن هزار و سیصد و هشتاد و دو ۱۲
output: 1382-11-02
source: دوازده آذر هزار و سیصد و هفتاد
output: 1370-09-12
source:  سه شهریور هزار و سیصد و هفتاد و پنج
output: 1375-06-03
source: هجده آذر 99
output: 1399-09-08


In [ ]:
prediction = mod.predict([source, s0, c0])
prediction = np.argmax(prediction, axis = -1)
prediction

array([[ 2],
       [ 4],
       [10],
       [10],
       [ 0],
       [ 1],
       [10],
       [ 0],
       [ 1],
       [ 9]])

In [ ]:
pip install git+https://github.com/Maluuba/nlg-eval.git@master

  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-req-build-w8trpqjd
  Running command git clone -q https://github.com/Maluuba/nlg-eval.git /tmp/pip-req-build-w8trpqjd
  Created wheel for nlg-eval: filename=nlg_eval-2.3-cp36-none-any.whl size=68175149 sha256=66f118e67f5c5c4f7141fcc1e91616a44b61b6e77e9e8bbd23e6405d2a7b8576
  Stored in directory: /tmp/pip-ephem-wheel-cache-mf969t0n/wheels/a5/7c/fd/f312beca2adcc3f49cb40570730658dad37bb5709f5d237a56
Successfully built nlg-eval


<div dir='rtl'style="text-align: right;font-family:verdana;">
<h1>ارزیابی مدل</h1>
<p>
</p>
</div>

In [ ]:
h=[]
m=[]
for i in testSet:
  h.append(i[0])
  m.append(i[1])

In [ ]:
import  nltk.translate.bleu_score as bleu
pr=[]
Example_dates=h
s0 = np.zeros((1, n_s))
c0 = np.zeros((1, n_s))
for example in Example_dates:
    
    source = string_to_int(example, Tx, human)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine[int(i)] for i in prediction]
    pr.append(output)

In [ ]:
str =[]
for j in range(len(pr)):
    str1 = ""    
    for ele in pr[j]:  
        str1 += ele
    str.append(str1)

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
belu اسکور نشان می دهد خروجی مدل تا چه اندازه با خروجی واقعی تفاوت داشته است که در این تمرین ما مقدار وزن برای همه نوع گرام برابر 25 صدم قرار دادیم.
بطور کلی بررسی که انجام می دهد روی یک حرف تا 4 حرف متوالی است که آیا درست بوده یا خیر که تا حدود 94 درصد است و نشان می دهد توالی خروجی تا حد خوبی با توالی اصلی برابر است و این بسیار خوب است.
</p>
</div>

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
a=0
for i in range(len(h)):
  reference = (m[i]).split()
  candidate = pr[i]
  f=sentence_bleu(reference, candidate ,weights=(0.25, 0.25, 0.25, 0.25))
  a=a+f
  #print(f)

bl=a/(len(h))
bl

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.9477709872024812

<div dir='rtl'style="text-align: right;font-family:verdana;">
<p>
دقت و بررسی داده هایی که نتایج خوب ندارند:


*   برای بررسی دقت راهکار دقیقی در کلاس معرفی نشده است و می توانیم روش های مختلفی را داشته باشیم مثلا تابع لاس که در قسمت قبل معرفی شد میزاین خطا را با توجه به تابع SparseCategoricalCrossentropy برای بردار احتمالات حروجی بر می گرداند و در هر پچ میانگیم می گیرد یا می توان آن ها که کاملا درست اند را در نظر گرفت و بررسی کرد چه تعدا خروجی کاملا درست داریم. ما می توانیم تمام روش ها را اعمال کنیم ولی با توجه به محدودیت وقت از  بررسی دقت با تمام روش ها صرف نظر می کنیم. فقط آن ها که کاملا با خروجی اصلی برابر بودند را می شماریم حدود 79.5 بوده است و این مقدار خوبی است .
*   ما آن ها که درست نبوده اند را چاپ کردیم و بنظر می رسد آن هایی که اعداد پشت سر هم بودند مدل بدتر عمل کرده است. البته در اکثر موارد تشخیص مدل کاملا غلط نبوده است و برای مثال در مواجه با 01 که عدد یک را برای روز مشخص کرده است عدد را 10 در نظر گرفته

۰۱ اسفند ۱۳۵۵<br>
input:  1355-12-01   output:  1355-12-10

7/22/86<br>
input:  1386-07-22   output:  1386-02-27

</p>
</div>

In [ ]:
c=0
for i in range(len(h)):
  reference = m[i]
  candidate = str[i]
  if reference==candidate:
    c=c+1
  else:
    print(h[i])
    print("input: ", reference,"  output: ",candidate)

ac=c/len(h)
ac

سهشنبه اردیبهشت ماه ۱۲ هزار و سیصد و هشتاد و هشت
input:  1388-02-12   output:  1388-02-21
۱/۲۲/۶۹
input:  1369-01-22   output:  1369-02-22
30.07.86
input:  1386-07-30   output:  1388-07-00
۱۲ آذر ۱۳۵۹
input:  1359-09-12   output:  1359-09-21
۱۳ آذر ۱۳۹۸
input:  1398-09-13   output:  1398-09-33
بیست و ششم دی هزار و چهارصد
input:  1400-10-26   output:  1400-16-26
شنبه آبان ماه 13 هزار و سیصد و نود و یک
input:  1391-08-13   output:  1391-08-31
۰۹.۱۰.۹۹
input:  1399-10-09   output:  1399-09-09
18.12.71
input:  1371-12-18   output:  1371-11-18
سی مرداد پنجاه
input:  1350-05-30   output:  1350-05-00
8/15/60
input:  1360-08-15   output:  1360-08-18
چهارشنبه خرداد ماه 25 1395
input:  1395-03-25   output:  1395-03-23
سوم دی ماه هزار و سیصد و نود و هفت
input:  1397-10-03   output:  1397-11-33
دو شنبه دی ماه ۳۱ شصت و سه
input:  1363-10-31   output:  1363-10-11
سهشنبه شهریور ماه 2 1350
input:  1350-06-02   output:  1350-06-20
مرداد ۳۱ ۱۳۶۵
input:  1365-05-31   output:  1365-05-33
6/1/81
input:  13

0.7954